# chips-net analysis

This notebook contains the full analysis of chipsnet showing the full path taken to the final version...

## Finding the best models
- [ ] [Exploring the data](#explore)
- [ ] [Which image representation is best?](#repr)
- [ ] [Which channel configuration is best?](#channel)
- [ ] [Which training sample is best?](#sample)
- [ ] [Which categorisation is best?](#cat)
- [ ] [Does vertex help cosmic classifier?](#cosmic)
- [ ] [Does primary count help beam classifier?](#beam)
- [ ] [Does lepton energy help neutrino energy estimation?](#energy)
- [ ] [Different sample energy estimation](#energy_samples)
- [ ] [Final performance and comparison with old reco/pid](#final)

## Explainability and resilience
- [ ] [Model calibration sensitivity: cosmic classification](#cosmic_calib)
- [ ] [Model calibration sensitivity: beam classification](#beam_calib)
- [ ] [Model calibration sensitivity: energy estimation](#energy_calib)
- [ ] [Clustering: cosmic classification](#cosmic_cluster)
- [ ] [Clustering: beam classification](#beam_cluster)
- [ ] [Explanation: cosmic classification](#cosmic_explain)
- [ ] [Explanation: beam classification](#beam_explain)
- [ ] [Explanation: energy estimation](#energy_explain)

In [ ]:
# Default input parameters for this notebook, they are overridden by 'papermill' at runtime if specified
config_path = "./config/analysis/eval.yaml"
save_path = "./data/output/"
nuel_map_file = "/unix/chips/jtingey/production/beam_all/nuel/map/chips_1200/selected_000_sim_map.root"
numu_map_file = "/unix/chips/jtingey/production/beam_all/numu/map/chips_1200/selected_000_sim_map.root"
cosmic_map_file = "/unix/chips/jtingey/production/cosmic_all/map/chips_1200/selected_000_sim_map.root"

In [ ]:
# Standard inports
import os
import logging
import importlib

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import mplhep as hep
import uproot
from scipy.stats import norm

# Setup Tensorflow with correct logging and GPU's for inference
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.disable(logging.CRITICAL)
gpus = tf.config.list_physical_devices('GPU')
if tf.config.list_physical_devices('GPU'):
    try:  # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:  # Memory growth must be set before GPUs have been initialized
        print(e)

import chipsnet.config
import chipsnet.data
import chipsnet.models
import chipsnet.utils
import chipsnet.plotting
from chipsnet.utils import process_ds

# Setup matplotlib style using mplhep
extra_styling = {
    "text.usetex": True,
    "pgf.texsystem": "pdflatex",
    "pgf.rcfonts": False,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    "axes.xmargin": 0
}
plt.style.use([hep.style.ROOT, extra_styling])

In [ ]:
# Reload method to import chipsnet modules on the fly if changes are made to them
config = chipsnet.config.get(config_path)
def reload():
    importlib.reload(chipsnet.config)
    importlib.reload(chipsnet.data)
    importlib.reload(chipsnet.models)
    importlib.reload(chipsnet.utils)
    importlib.reload(chipsnet.plotting)
    config = chipsnet.config.get(config_path)

# Finding the best models

<a id='explore'></a>

## Exploring the data (explore)
* 1
* 2
* 3

In [ ]:
both_ev, both_out = process_ds(config, "both_cosmic")
flux_ev, flux_out = process_ds(config, "flux_cosmic")
uniform_ev, uniform_out = process_ds(config, "uniform_cosmic")

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(18, 5), gridspec_kw={'hspace': 0.3})
cats = np.arange(5)
axs.bar(
    cats, 
    [len(both_ev[(both_ev.t_nu_type == 0) & (both_ev.t_sign_type == 0) & (both_ev.t_cosmic_cat == 0)]), 
     len(both_ev[(both_ev.t_nu_type == 0) & (both_ev.t_sign_type == 1) & (both_ev.t_cosmic_cat == 0)]), 
     len(both_ev[(both_ev.t_nu_type == 1) & (both_ev.t_sign_type == 0) & (both_ev.t_cosmic_cat == 0)]), 
     len(both_ev[(both_ev.t_nu_type == 1) & (both_ev.t_sign_type == 1) & (both_ev.t_cosmic_cat == 0)]), 
     len(both_ev[both_ev.t_cosmic_cat == 1])],
    color="firebrick")
axs.set_xticks(cats)
axs.set_xticklabels(('$\\nu_{e}$', '$\\bar{\\nu_{e}}$', '$\\nu_{\mu}$', '$\\bar{\\nu_{\mu}}$', 'cosmic muons'))
axs.set_ylabel("Test events")
chipsnet.plotting.save(save_path + "nu_types")

In [ ]:
data_both = [
    len(both_ev[both_ev["t_nu_nc_cat"] == i])
    for i in range(len(chipsnet.data.MAP_NU_NC_COMB_CAT["labels"]))
]
cats = np.arange(len(chipsnet.data.MAP_NU_NC_COMB_CAT["labels"]))
width = 0.45

fig, axs = plt.subplots(1, 1, figsize=(18, 5), gridspec_kw={"hspace": 0.3})
axs.bar(cats, data_both, color="royalblue", width=width, label="training sample")
axs.set_xticks(cats)
axs.set_xticklabels(chipsnet.data.MAP_NU_NC_COMB_CAT["labels"], fontsize=14, rotation="vertical")
axs.set_ylabel("Testing events")
axs.set_yscale('log')
chipsnet.plotting.save(save_path + "training_sample")

In [ ]:
chipsnet.plotting.plot_cats(uniform_ev, flux_ev, chipsnet.data.MAP_INT_TYPE, save_path)

In [ ]:
chipsnet.plotting.plot_cats(uniform_ev, flux_ev, chipsnet.data.MAP_ALL_CAT, save_path)

In [ ]:
chipsnet.plotting.plot_cats(uniform_ev, flux_ev, chipsnet.data.MAP_FULL_COMB_CAT, save_path)

In [ ]:
chipsnet.plotting.plot_cats(uniform_ev, flux_ev, chipsnet.data.MAP_NU_NC_COMB_CAT, save_path)

In [ ]:
chipsnet.plotting.plot_cats(uniform_ev, flux_ev, chipsnet.data.MAP_NC_COMB_CAT, save_path)

In [ ]:
line_g = matplotlib.lines.Line2D([], [], c='green')
line_r = matplotlib.lines.Line2D([], [], c='red')

fig, axs = plt.subplots(1, 3, figsize=(18, 5), gridspec_kw={'hspace': 0.3})
axs[0].hist(flux_ev["t_vtx_x"]/100, range=(-12.5, 12.5), bins=25, color='green', histtype='step')
axs[0].hist(flux_ev["r_vtx_x"]*1250/100, range=(-12.5, 12.5), bins=25, color='red', histtype='step')
#axs[0].legend(loc="upper left", handles=[line_g, line_r], labels=["true", "reco"])
axs[0].set_xlabel('Vertex x-position (m)', fontsize=20)
axs[0].set_ylabel('Frequency', fontsize=20)

axs[1].hist(flux_ev["t_vtx_y"]/100, range=(-12.5, 12.5), bins=25, color='green', histtype='step')
axs[1].hist(flux_ev["r_vtx_y"]*1250/100, range=(-12.5, 12.5), bins=25, color='red', histtype='step')
#axs[1].legend(loc="upper left", handles=[line_g, line_r], labels=["true", "reco"])
axs[1].set_xlabel('Vertex y-position (m)', fontsize=20)

axs[2].hist(flux_ev["t_vtx_z"]/100, range=(-6, 10), bins=25, color='green', histtype='step')
axs[2].hist(flux_ev["r_vtx_z"]*600/100, range=(-6, 10), bins=25, color='red', histtype='step')
#axs[2].legend(loc="upper left", handles=[line_g, line_r], labels=["true", "reco"])
axs[2].set_xlabel('Vertex z-position (m)', fontsize=20)
chipsnet.plotting.save(save_path + "vtx_positions")

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5), gridspec_kw={'hspace': 0.3})
axs[0].hist(flux_ev["t_vtx_x"]/100-(flux_ev["r_vtx_x"]*1250/100), range=(-7, 2), bins=50, color='black', histtype='step')
axs[0].set_xlabel('Vertex true-reco x-position (m)', fontsize=20)
axs[0].set_ylabel('Frequency', fontsize=20)

axs[1].hist(flux_ev["t_vtx_y"]/100-(flux_ev["r_vtx_y"]*1250/100), range=(-2, 2), bins=50, color='black', histtype='step')
axs[1].set_xlabel('Vertex true-reco y-position (m)', fontsize=20)

axs[2].hist(flux_ev["t_vtx_z"]/100-(flux_ev["r_vtx_z"]*600/100), range=(-2, 8), bins=50, color='black', histtype='step')
axs[2].set_xlabel('Vertex true-reco z-position (m)', fontsize=20)
chipsnet.plotting.save(save_path + "true_reco_vtx")

In [ ]:
line_g = matplotlib.lines.Line2D([], [], c='green')
line_b = matplotlib.lines.Line2D([], [], c='blue')
line_r = matplotlib.lines.Line2D([], [], c='red')

fig, axs = plt.subplots(1, 1, figsize=(8, 5), gridspec_kw={'hspace': 0.3})
axs.hist(flux_ev[flux_ev["t_comb_cat"] == 0]["t_nu_energy"]/1000, weights=flux_ev[flux_ev["t_comb_cat"] == 0]["w"],
         range=(0, 10), bins=40, color='green', histtype='step', density=True)
axs.hist(flux_ev[flux_ev["t_comb_cat"] == 1]["t_nu_energy"]/1000, weights=flux_ev[flux_ev["t_comb_cat"] == 1]["w"],
         range=(0, 10), bins=40, color='blue', histtype='step', density=True)
axs.hist(flux_ev[flux_ev["t_comb_cat"] == 2]["t_nu_energy"]/1000, weights=flux_ev[flux_ev["t_comb_cat"] == 2]["w"],
         range=(0, 10), bins=40, color='red', histtype='step', density=True)
axs.legend(loc="upper left", handles=[line_g, line_b, line_r], labels=["$\\nu_{e}$ CC", "$\\nu_{\mu}$ CC", "NC"])
axs.set_xlabel('Neutrino energy (GeV)', fontsize=20)
axs.set_ylabel('Fraction of events', fontsize=20)
chipsnet.plotting.save(save_path + "nu_energies")

In [ ]:
# Setup an images dictionary to hold example images
image_size = [64, 64]
images_dict = {"r_raw_charge_map_origin": [], "r_raw_charge_map_iso": [], "r_raw_charge_map_vtx": [],
               "r_raw_time_map_origin": [], "r_raw_time_map_iso": [], "r_raw_time_map_vtx": [], "r_raw_hit_hough_map_vtx": []}
               
# Load the images into the dictionary
nuel_file = uproot.open(nuel_map_file)
numu_file = uproot.open(numu_map_file)
cosmic_file = uproot.open(cosmic_map_file)
for image_name in images_dict.keys():
    images_dict[image_name].extend(nuel_file['reco'].array(image_name))
    images_dict[image_name].extend(numu_file['reco'].array(image_name))
    images_dict[image_name].extend(cosmic_file['reco'].array(image_name))

# Calculate fraction of hits in the endcaps
cap_total = 0
barrel_total = 0
for event in images_dict['r_raw_charge_map_vtx']:
    sections = np.split(event, [18, 46], axis=0)
    cap_total += (sections[0].sum() + sections[2].sum())
    barrel_total += sections[1].sum()

# This is with cosmic events included
print("Fraction of deposited charge in endcaps = {}".format(cap_total/(cap_total+barrel_total)))

In [ ]:
chipsnet.plotting.plot_hit_time(images_dict, 1, save_path)

In [ ]:
chipsnet.plotting.plot_hough(images_dict, 1, save_path)

In [ ]:
chipsnet.plotting.plot_8bit_range(images_dict, save_path=save_path)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 10), gridspec_kw={'hspace': 0.3})
axs[0, 0].hist(flux_ev[flux_ev.t_comb_cat == 0]['r_raw_total_digi_q'], color='green',
               histtype='step', range=(0, 5000), bins=40, density=True, label="$\\nu_{e}$ CC")
axs[0, 0].hist(flux_ev[flux_ev.t_comb_cat == 1]['r_raw_total_digi_q'], color='blue', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="$\\nu_{\mu}$ CC")
axs[0, 0].hist(flux_ev[flux_ev.t_comb_cat == 2]['r_raw_total_digi_q'], color='red', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="NC")
axs[0, 0].hist(flux_ev[flux_ev.t_comb_cat == 3]['r_raw_total_digi_q'], color='black', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="Cosmic")
axs[0, 0].set(xlabel='Total collected charge (p.e)', ylabel='Frequency')
axs[0, 0].axvspan(0, config.eval.cuts.q, alpha=0.5, color='grey')
axs[0, 0].legend()

axs[0, 1].hist(flux_ev[flux_ev.t_comb_cat == 0]['r_first_ring_height'], color='green',
               histtype='step', range=(0, 5000), bins=40, density=True, label="$\\nu_{e}$ CC")
axs[0, 1].hist(flux_ev[flux_ev.t_comb_cat == 1]['r_first_ring_height'], color='blue', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="$\\nu_{\mu}$ CC")
axs[0, 1].hist(flux_ev[flux_ev.t_comb_cat == 2]['r_first_ring_height'], color='red', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="NC")
axs[0, 1].hist(flux_ev[flux_ev.t_comb_cat == 3]['r_first_ring_height'], color='black', 
               histtype='step', range=(0, 5000), bins=40, density=True, label="Cosmic")
axs[0, 1].set(xlabel='Hough ring height (p.e)', ylabel='Frequency')
axs[0, 1].axvspan(0, config.eval.cuts.h, alpha=0.5, color='grey')
axs[0, 1].legend()

axs[1, 0].hist(flux_ev[flux_ev.t_comb_cat == 0]['r_dirTheta'], color='green',
               histtype='step', range=(-1, 1), bins=64, density=True, label="$\\nu_{e}$ CC")
axs[1, 0].hist(flux_ev[flux_ev.t_comb_cat == 1]['r_dirTheta'], color='blue', 
               histtype='step', range=(-1, 1), bins=64, density=True, label="$\\nu_{\mu}$ CC")
axs[1, 0].hist(flux_ev[flux_ev.t_comb_cat == 2]['r_dirTheta'], color='red', 
               histtype='step', range=(-1, 1), bins=64, density=True, label="NC")
axs[1, 0].hist(flux_ev[flux_ev.t_comb_cat == 3]['r_dirTheta'], color='black', 
               histtype='step', range=(-1, 1), bins=64, density=True, label="Cosmic")
axs[1, 0].set(xlabel='Reco theta direction (cos($\\theta$))', ylabel='Frequency')
axs[1, 0].axvspan(-1, -config.eval.cuts.theta, alpha=0.5, color='grey')
axs[1, 0].axvspan(config.eval.cuts.theta, 1, alpha=0.5, color='grey')
axs[1, 0].legend()

axs[1, 1].hist(flux_ev[flux_ev.t_comb_cat == 0]['r_dirPhi']*3.14159, color='green',
               histtype='step', range=(-3.2, 3.2), bins=64, density=True, label="$\\nu_{e}$ CC")
axs[1, 1].hist(flux_ev[flux_ev.t_comb_cat == 1]['r_dirPhi']*3.14159, color='blue', 
               histtype='step', range=(-3.2, 3.2), bins=64, density=True, label="$\\nu_{\mu}$ CC")
axs[1, 1].hist(flux_ev[flux_ev.t_comb_cat == 2]['r_dirPhi']*3.14159, color='red', 
               histtype='step', range=(-3.2, 3.2), bins=64, density=True, label="NC")
axs[1, 1].hist(flux_ev[flux_ev.t_comb_cat == 3]['r_dirPhi']*3.14159, color='black', 
               histtype='step', range=(-3.2, 3.2), bins=64, density=True, label="Cosmic")
axs[1, 1].set(xlabel='Reco phi direction (radians)', ylabel='Frequency')
axs[1, 1].axvspan(-3.2, -config.eval.cuts.phi*3.14159, alpha=0.5, color='grey')
axs[1, 1].axvspan(config.eval.cuts.phi*3.14159, 3.2, alpha=0.5, color='grey')
axs[1, 1].legend()
chipsnet.plotting.save(save_path + "simple_cuts")

In [ ]:
# Tidy up
del both_ev, both_out
del flux_ev, flux_out
del uniform_ev, uniform_out
del images_dict

<a id='repr'></a>

## Which image representation is best? (repr)
* 1
* 2
* 3

In [ ]:
v_ev, v_out = process_ds(config, "v_ct_flux", ["repr_v_ct"])  # Vertex theta/phi repr
o_ev, o_out = process_ds(config, "o_ct_flux", ["repr_o_ct"])  # Origin theta/phi repr
i_ev, i_out = process_ds(config, "i_ct_flux", ["repr_i_ct"])  # Origin x+/x- repr

In [ ]:
chipsnet.plotting.plot_curves([v_out, o_out, i_out], save_path+"repr_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([v_out, o_out, i_out], v_ev, save_path+"repr_hists")

In [ ]:
chipsnet.plotting.plot_history_comparison(config, ["repr_v_ct", "repr_o_ct", "repr_i_ct"], save_path+"repr_history")

In [ ]:
chipsnet.utils.print_output_comparison([v_out, o_out, i_out])

In [ ]:
# Tidy up
del v_ev, v_out
del o_ev, o_out
del i_ev, i_out

<a id='channel'></a>

## Which channel configuration is best? (channel)
* 1
* 2
* 3

In [ ]:
c_ev, c_out = process_ds(config, "v_c_flux", ["repr_v_c"])  # Just charge channel
ct_ev, ct_out = process_ds(config, "v_ct_flux", ["repr_v_ct"])  # Charge+time channels
cth_ev, cth_out = process_ds(config,"v_cth_flux", ["repr_v_cth"])  # Charge+time+hough channels

In [ ]:
chipsnet.plotting.plot_curves([c_out, ct_out, cth_out], save_path+"channel_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([c_out, ct_out, cth_out], c_ev, save_path+"channel_hists")

In [ ]:
chipsnet.plotting.plot_history_comparison(config, ["repr_v_c", "repr_v_ct", "repr_v_cth"], save_path+"channel_history")

In [ ]:
chipsnet.utils.print_output_comparison([c_out, ct_out, cth_out])

In [ ]:
# Tidy up
del c_ev, c_out
del ct_ev, ct_out
del cth_ev, cth_out

<a id='sample'></a>

## Which training sample is best? (sample)
* 1
* 2
* 3

In [ ]:
b_ev, b_out = process_ds(config, "flux", ["sample_both"])  # Both: flux and uniform sample
f_ev, f_out = process_ds(config, "flux", ["sample_flux"])  # Flux sample
u_ev, u_out = process_ds(config, "flux", ["sample_uniform"])  # Uniform sample

In [ ]:
chipsnet.plotting.plot_curves([b_out, f_out, u_out], save_path+"sample_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([b_out, f_out, u_out], b_ev, save_path+"sample_hists")

In [ ]:
chipsnet.plotting.plot_history_comparison(config, ["sample_both", "sample_flux", "sample_uniform"], save_path+"sample_history")

In [ ]:
chipsnet.utils.print_output_comparison([b_out, f_out, u_out])

In [ ]:
chipsnet.plotting.plot_combined_values(b_ev, "sample_both_", save_path+"sample_both_values")

In [ ]:
chipsnet.plotting.plot_combined_values(f_ev, "sample_flux_", save_path+"sample_flux_values")

In [ ]:
chipsnet.plotting.plot_combined_values(u_ev, "sample_uniform_", save_path+"sample_uniform_values")

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(b_out["all_matrices"][0], annot=False, vmin=0.0, vmax=1.0)
plt.xlabel('True', fontsize = 30)
plt.ylabel('Predicted', fontsize = 30)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(f_out["all_matrices"][0], annot=False, vmin=0.0, vmax=1.0)
plt.xlabel('True', fontsize = 30)
plt.ylabel('Predicted', fontsize = 30)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(u_out["all_matrices"][0], annot=False, vmin=0.0, vmax=1.0)
plt.xlabel('True', fontsize = 30)
plt.ylabel('Predicted', fontsize = 30)
plt.show()

In [ ]:
# Tidy up
del b_ev, b_out
del f_ev, f_out
del u_ev, u_out

<a id='cat'></a>

## Which categorisation is best? (cat)
* 1
* 2
* 3

In [ ]:
# t_final_cat model
final_ev, final_out = process_ds(config, "flux", ["cat_t_final_cat"], ["t_final_cat"])
# t_all_cat model
all_ev, all_out = process_ds(config, "flux", ["cat_t_all_cat"], ["t_all_cat"])
# t_comb_cat model
comb_ev, comb_out = process_ds(config, "flux", ["cat_t_comb_cat"], ["t_comb_cat"])
# t_nu_nc_cat model
nu_nc_ev, nu_nc_out = process_ds(config, "flux", ["cat_t_nu_nc_cat"], ["t_nu_nc_cat"])
# t_nc_cat model
nc_ev, nc_out = process_ds(config, "flux", ["cat_t_nc_cat"], ["t_nc_cat"])
# ["t_comb_cat", "t_cc_type", "t_nc_type"] model
split_ev, split_out = process_ds(config, "flux", ["cat_split"], ["t_comb_cat"])
# ["t_comb_cat", "t_cc_type", "t_nc_type"] model, with learnt multitask weights
split_learn_ev, split_learn_out = process_ds(config, "flux", ["cat_split_learn"], ["t_comb_cat"])  

In [ ]:
chipsnet.plotting.plot_curves([nu_nc_out, all_out, final_out, split_out], save_path+"cat_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([nu_nc_out, all_out, final_out, split_out], nu_nc_ev, save_path+"cat_hists")

In [ ]:
chipsnet.utils.print_output_comparison([final_out, all_out, comb_out, nu_nc_out, nc_out, split_out, split_learn_out])

In [ ]:
# Tidy up
del final_ev, final_out
del all_ev, all_out
del comb_ev, comb_out
del nu_nc_ev, nu_nc_out
del nc_ev, nc_out
del split_ev, split_out
del split_learn_ev, split_learn_out

<a id='cosmic'></a>

## Does vertex help cosmic classifier? (cosmic)
* 1
* 2
* 3

In [ ]:
# t_cosmic_cat model
c_ev, c_out = process_ds(config,"flux_cosmic", ["cosmic"])
# ["t_cosmic_cat", "t_vtx_x", "t_vtx_y", "t_vtx_z", "t_vtx_t"] model
c_vtx_ev, c_vtx_out = process_ds(config, "flux_cosmic", ["cosmic_vtx"])
# ["t_cosmic_cat", "t_vtx_x", "t_vtx_y", "t_vtx_z", "t_vtx_t"] model, with learnt multitask weights
c_vtx_learn_ev, c_vtx_learn_out = process_ds(config, "flux_cosmic", ["cosmic_vtx_learn"])

In [ ]:
# Tidy up
del c_ev, c_out
del c_vtx_ev, c_vtx_out
del c_vtx_learn_ev, c_vtx_learn_out

<a id='beam'></a>

## Does primary count help beam classifier? (beam)
* 1
* 2
* 3

In [ ]:
# t_nu_nc_cat model
b_ev, b_out = process_ds(config, "flux", ["beam"])
# ["t_nu_nc_cat", "prim_p", "prim_cp", "prim_np"] model
b_prim_ev, b_prim_out = process_ds(config, "flux", ["beam_primaries"])
# ["t_nu_nc_cat", "prim_p", "prim_cp", "prim_np"] model, with learnt multitask weights
b_prim_learn_ev, b_prim_learn_out = process_ds(config, "flux", ["beam_primaries_learn"])

In [ ]:
chipsnet.plotting.plot_curves([b_out, b_prim_out, b_prim_learn_out], save_path+"beam_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([b_out, b_prim_out, b_prim_learn_out], beam_ev, save_path+"beam_hists")

In [ ]:
chipsnet.utils.print_output_comparison([b_out, b_prim_out, b_prim_learn_out])

In [ ]:
# Tidy up
del b_ev, b_out
del b_prim_ev, b_prim_out
del b_prim_learn_ev, b_prim_learn_out

<a id='energy'></a>

## Does lepton energy help neutrino energy estimation? (energy)
* 1
* 2
* 3

In [ ]:
# t_nu_energy model
e_ev, e_out = process_ds(config, "flux", ["energy"])
# ["t_nu_energy", "t_lep_energy"] model
e_lep_ev, e_lep_out = process_ds(config, "flux", ["energy_lepton"])
# # ["t_nu_energy", "t_lep_energy"] model, with learnt multitask weights
e_lep_learn_ev, e_lep_learn_out = process_ds(config, "flux", ["energy_lepton_learn"])

In [ ]:
# Tidy up
del e_ev, e_out
del e_lep_ev, e_lep_out
del e_lep_learn_ev, e_lep_learn_out

<a id='energy_samples'></a>

## Different sample energy estimation (energy_samples)
* 1
* 2
* 3

In [ ]:
# All the different interaction type samples and associated models
nuel_cccoh_ev = chipsnet.utils.predict_energies(config, "nuel_cccoh", ["nuel_cccoh_energy"])
nuel_ccdis_ev = chipsnet.utils.predict_energies(config, "nuel_ccdis", ["nuel_ccdis_energy"])
nuel_ccqel_ev = chipsnet.utils.predict_energies(config, "nuel_ccqel", ["nuel_ccqel_energy"])
nuel_ccres_ev = chipsnet.utils.predict_energies(config, "nuel_ccres", ["nuel_ccres_energy"])
nuel_ccmec_ev = chipsnet.utils.predict_energies(config, "nuel_ccmec", ["nuel_ccmec_energy"])
nuel_ccqelmec_ev = chipsnet.utils.predict_energies(config, "nuel_ccqelmec", ["nuel_ccqelmec_energy"])
numu_cccoh_ev = chipsnet.utils.predict_energies(config, "numu_cccoh", ["numu_cccoh_energy"])
numu_ccdis_ev = chipsnet.utils.predict_energies(config, "numu_ccdis", ["numu_ccdis_energy"])
numu_ccqel_ev = chipsnet.utils.predict_energies(config, "numu_ccqel", ["numu_ccqel_energy"])
numu_ccres_ev = chipsnet.utils.predict_energies(config, "numu_ccres", ["numu_ccres_energy"])
numu_ccmec_ev = chipsnet.utils.predict_energies(config, "numu_ccmec", ["numu_ccmec_energy"])
numu_ccqelmec_ev = chipsnet.utils.predict_energies(config, "numu_ccqelmec", ["numu_ccqelmec_energy"])
flux_ev = chipsnet.utils.predict_energies(config, "flux", ["nuel_cc_energy", "numu_cc_energy", "nc_energy"])

In [ ]:
e_range = [[0, 8000], [0, 8000]]
e_bins = [40, 40]
fig, axs = plt.subplots(5, 2, figsize=(10, 20), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
axs[0, 0].hist2d(nuel_cccoh_ev["t_nuEnergy"], nuel_cccoh_ev["nuel_cccoh_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[0, 0].grid()
axs[0, 0].label_outer()
axs[0, 0].set(ylabel=r"Reco energy (MeV)")
axs[0, 0].set_title(r"nuel")
axs[1, 0].hist2d(nuel_ccdis_ev["t_nuEnergy"], nuel_ccdis_ev["nuel_ccdis_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[1, 0].grid()
axs[1, 0].label_outer()
axs[1, 0].set(ylabel=r"Reco energy (MeV)")
axs[2, 0].hist2d(nuel_ccqel_ev["t_nuEnergy"], nuel_ccqel_ev["nuel_ccqel_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[2, 0].grid()
axs[2, 0].label_outer()
axs[2, 0].set(ylabel=r"Reco energy (MeV)")
axs[3, 0].hist2d(nuel_ccres_ev["t_nuEnergy"], nuel_ccres_ev["nuel_ccres_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[3, 0].grid()
axs[3, 0].label_outer()
axs[3, 0].set(ylabel=r"Reco energy (MeV)")
axs[4, 0].hist2d(nuel_ccmec_ev["t_nuEnergy"], nuel_ccmec_ev["nuel_ccmec_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[4, 0].grid()
axs[4, 0].label_outer()
axs[4, 0].set(xlabel=r"True energy (MeV)", ylabel=r"Reco energy (MeV)")
axs[0, 1].hist2d(numu_cccoh_ev["t_nuEnergy"], numu_cccoh_ev["numu_cccoh_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[0, 1].grid()
axs[0, 1].label_outer()
axs[0, 1].set_title(r"numu")
axs[0, 1].text(8000, 2250, "CC-COH", rotation=-90)
axs[1, 1].hist2d(numu_ccdis_ev["t_nuEnergy"], numu_ccdis_ev["numu_ccdis_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[1, 1].grid()
axs[1, 1].label_outer()
axs[1, 1].text(8000, 2250, "CC-DIS", rotation=-90)
axs[2, 1].hist2d(numu_ccqel_ev["t_nuEnergy"], numu_ccqel_ev["numu_ccqel_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[2, 1].grid()
axs[2, 1].label_outer()
axs[2, 1].text(8000, 2250, "CC-QEL", rotation=-90)
axs[3, 1].hist2d(numu_ccres_ev["t_nuEnergy"], numu_ccres_ev["numu_ccres_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[3, 1].grid()
axs[3, 1].label_outer()
axs[3, 1].text(8000, 2250, "CC-RES", rotation=-90)
axs[4, 1].hist2d(numu_ccmec_ev["t_nuEnergy"], numu_ccmec_ev["numu_ccmec_energy_pred_t_nuEnergy"],
                 range=e_range, bins=e_bins, cmap="Reds")
axs[4, 1].grid()
axs[4, 1].label_outer()
axs[4, 1].set(xlabel=r"True energy (MeV)")
axs[4, 1].text(8000, 2250, "CC-MEC", rotation=-90)
chipsnet.plotting.save(save_path + "true_vs_reco_energy_cc")

In [ ]:
e_range = [[0, 8000], [0, 8000]]
e_bins = [40, 40]
fig, axs = plt.subplots(1, 3, figsize=(15, 4), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
axs[0].hist2d(flux_ev[flux_ev["t_comb_cat"] == 0]["t_nuEnergy"], flux_ev[flux_ev["t_comb_cat"] == 0]["nuel_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=flux_ev[flux_ev["t_comb_cat"] == 0]["w"], cmap="Reds")
axs[0].grid()
axs[0].label_outer()
axs[0].set(xlabel=r"True energy (MeV)", ylabel=r"Reco energy (MeV)")
axs[0].set_title(r"Nuel CC")
axs[1].hist2d(flux_ev[flux_ev["t_comb_cat"] == 1]["t_nuEnergy"], flux_ev[flux_ev["t_comb_cat"] == 1]["numu_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=flux_ev[flux_ev["t_comb_cat"] == 1]["w"], cmap="Reds")
axs[1].grid()
axs[1].label_outer()
axs[1].set(xlabel=r"True energy (MeV)")
axs[1].set_title(r"Numu CC")
axs[2].hist2d(flux_ev[flux_ev["t_comb_cat"] == 2]["t_nuEnergy"], flux_ev[flux_ev["t_comb_cat"] == 2]["nc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=flux_ev[flux_ev["t_comb_cat"] == 2]["w"], cmap="Reds")
axs[2].grid()
axs[2].label_outer()
axs[2].set(xlabel=r"True energy (MeV)")
axs[2].set_title(r"NC")
chipsnet.plotting.save(save_path + "true_vs_reco_energy_comb")

In [ ]:
nuel_cccoh_ev['true_reco_frac_nuEnergy'] = (nuel_cccoh_ev["t_nuEnergy"] - nuel_cccoh_ev["nuel_cccoh_energy_pred_t_nuEnergy"]) / nuel_cccoh_ev["t_nuEnergy"]
nuel_ccdis_ev['true_reco_frac_nuEnergy'] = (nuel_ccdis_ev["t_nuEnergy"] - nuel_ccdis_ev["nuel_ccdis_energy_pred_t_nuEnergy"]) / nuel_ccdis_ev["t_nuEnergy"]
nuel_ccqel_ev['true_reco_frac_nuEnergy'] = (nuel_ccqel_ev["t_nuEnergy"] - nuel_ccqel_ev["nuel_ccqel_energy_pred_t_nuEnergy"]) / nuel_ccqel_ev["t_nuEnergy"]
nuel_ccres_ev['true_reco_frac_nuEnergy'] = (nuel_ccres_ev["t_nuEnergy"] - nuel_ccres_ev["nuel_ccres_energy_pred_t_nuEnergy"]) / nuel_ccres_ev["t_nuEnergy"]
nuel_ccmec_ev['true_reco_frac_nuEnergy'] = (nuel_ccmec_ev["t_nuEnergy"] - nuel_ccmec_ev["nuel_ccmec_energy_pred_t_nuEnergy"]) / nuel_ccmec_ev["t_nuEnergy"]
numu_cccoh_ev['true_reco_frac_nuEnergy'] = (numu_cccoh_ev["t_nuEnergy"] - numu_cccoh_ev["numu_cccoh_energy_pred_t_nuEnergy"]) / numu_cccoh_ev["t_nuEnergy"]
numu_ccdis_ev['true_reco_frac_nuEnergy'] = (numu_ccdis_ev["t_nuEnergy"] - numu_ccdis_ev["numu_ccdis_energy_pred_t_nuEnergy"]) / numu_ccdis_ev["t_nuEnergy"]
numu_ccqel_ev['true_reco_frac_nuEnergy'] = (numu_ccqel_ev["t_nuEnergy"] - numu_ccqel_ev["numu_ccqel_energy_pred_t_nuEnergy"]) / numu_ccqel_ev["t_nuEnergy"]
numu_ccres_ev['true_reco_frac_nuEnergy'] = (numu_ccres_ev["t_nuEnergy"] - numu_ccres_ev["numu_ccres_energy_pred_t_nuEnergy"]) / numu_ccres_ev["t_nuEnergy"]
numu_ccmec_ev['true_reco_frac_nuEnergy'] = (numu_ccmec_ev["t_nuEnergy"] - numu_ccmec_ev["numu_ccmec_energy_pred_t_nuEnergy"]) / numu_ccmec_ev["t_nuEnergy"]

In [ ]:
def frac_e_vs_e(events, low=500, high=8000, bin_size=500):
    e_list, std_list = [], []
    for energy in range(low, high, bin_size):
        subset = events[(events["t_nuEnergy"] >= energy)]
        subset = subset[(events["t_nuEnergy"] <= (energy + bin_size))]
        mu, std = norm.fit(subset['true_reco_frac_nuEnergy'])
        std_list.append(std)
        e_list.append(energy+(bin_size/2))
    return (e_list, std_list)

def frac_e_vs_vtxX(events, low=-1250, high=1250, bin_size=100):
    vtx_list, std_list = [], []
    for vtx in range(low, high, bin_size):
        subset = events[(events["t_vtxX"] >= vtx)]
        subset = subset[(events["t_vtxX"] <= (vtx + bin_size))]
        mu, std = norm.fit(subset['true_reco_frac_nuEnergy'])
        std_list.append(std)
        vtx_list.append(vtx+(bin_size/2))
    return (vtx_list, std_list)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10, 20), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
e_list, std_list = frac_e_vs_e(nuel_cccoh_ev)
axs[0, 0].plot(e_list, std_list)
axs[0, 0].label_outer()
axs[0, 0].set(ylabel=r"(True-reco)/true E")
axs[0, 0].set_title(r"nuel")
axs[0, 0].set_ylim(0, 0.5)
axs[0, 0].grid()
e_list, std_list = frac_e_vs_e(nuel_ccdis_ev)
axs[1, 0].plot(e_list, std_list)
axs[1, 0].label_outer()
axs[1, 0].set(ylabel=r"(True-reco)/true E")
axs[1, 0].set_ylim(0, 0.5)
axs[1, 0].grid()
e_list, std_list = frac_e_vs_e(nuel_ccqel_ev)
axs[2, 0].plot(e_list, std_list)
axs[2, 0].label_outer()
axs[2, 0].set(ylabel=r"(True-reco)/true E")
axs[2, 0].set_ylim(0, 0.5)
axs[2, 0].grid()
e_list, std_list = frac_e_vs_e(nuel_ccres_ev)
axs[3, 0].plot(e_list, std_list)
axs[3, 0].label_outer()
axs[3, 0].set(ylabel=r"(True-reco)/true E")
axs[3, 0].set_ylim(0, 0.5)
axs[3, 0].grid()
e_list, std_list = frac_e_vs_e(nuel_ccmec_ev)
axs[4, 0].plot(e_list, std_list)
axs[4, 0].label_outer()
axs[4, 0].set(xlabel=r"Neutrino energy (MeV)", ylabel=r"(True-reco)/true E")
axs[4, 0].set_ylim(0, 0.5)
axs[4, 0].grid()
e_list, std_list = frac_e_vs_e(numu_cccoh_ev)
axs[0, 1].plot(e_list, std_list)
axs[0, 1].label_outer()
axs[0, 1].set_title(r"numu")
axs[0, 1].text(8000, 0.15, "CC-COH", rotation=-90)
axs[0, 1].set_ylim(0, 0.5)
axs[0, 1].grid()
e_list, std_list = frac_e_vs_e(numu_ccdis_ev)
axs[1, 1].plot(e_list, std_list)
axs[1, 1].label_outer()
axs[1, 1].text(8000, 0.15, "CC-DIS", rotation=-90)
axs[1, 1].set_ylim(0, 0.5)
axs[1, 1].grid()
e_list, std_list = frac_e_vs_e(numu_ccqel_ev)
axs[2, 1].plot(e_list, std_list)
axs[2, 1].label_outer()
axs[2, 1].text(8000, 0.15, "CC-QEL", rotation=-90)
axs[2, 1].set_ylim(0, 0.5)
axs[2, 1].grid()
e_list, std_list = frac_e_vs_e(numu_ccres_ev)
axs[3, 1].plot(e_list, std_list)
axs[3, 1].label_outer()
axs[3, 1].text(8000, 0.15, "CC-RES", rotation=-90)
axs[3, 1].set_ylim(0, 0.5)
axs[3, 1].grid()
e_list, std_list = frac_e_vs_e(numu_ccmec_ev)
axs[4, 1].plot(e_list, std_list)
axs[4, 1].label_outer()
axs[4, 1].set(xlabel=r"Neutrino energy (MeV)")
axs[4, 1].text(8000, 0.15, "CC-MEC", rotation=-90)
axs[4, 1].set_ylim(0, 0.5)
axs[4, 1].grid()
chipsnet.plotting.save(save_path + "true_vs_frac_energy")

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10, 20), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
e_list, std_list = frac_e_vs_vtxX(nuel_cccoh_ev)
axs[0, 0].plot(e_list, std_list)
axs[0, 0].label_outer()
axs[0, 0].set(ylabel=r"(True-reco)/true E")
axs[0, 0].set_title(r"nuel")
axs[0, 0].set_ylim(0, 0.5)
axs[0, 0].grid()
e_list, std_list = frac_e_vs_vtxX(nuel_ccdis_ev)
axs[1, 0].plot(e_list, std_list)
axs[1, 0].label_outer()
axs[1, 0].set(ylabel=r"(True-reco)/true E")
axs[1, 0].set_ylim(0, 0.5)
axs[1, 0].grid()
e_list, std_list = frac_e_vs_vtxX(nuel_ccqel_ev)
axs[2, 0].plot(e_list, std_list)
axs[2, 0].label_outer()
axs[2, 0].set(ylabel=r"(True-reco)/true E")
axs[2, 0].set_ylim(0, 0.5)
axs[2, 0].grid()
e_list, std_list = frac_e_vs_vtxX(nuel_ccres_ev)
axs[3, 0].plot(e_list, std_list)
axs[3, 0].label_outer()
axs[3, 0].set(ylabel=r"(True-reco)/true E")
axs[3, 0].set_ylim(0, 0.5)
axs[3, 0].grid()
e_list, std_list = frac_e_vs_vtxX(nuel_ccmec_ev)
axs[4, 0].plot(e_list, std_list)
axs[4, 0].label_outer()
axs[4, 0].set(xlabel=r"Vertex x-position (cm)", ylabel=r"(True-reco)/true E")
axs[4, 0].set_ylim(0, 0.5)
axs[4, 0].grid()
e_list, std_list = frac_e_vs_vtxX(numu_cccoh_ev)
axs[0, 1].plot(e_list, std_list)
axs[0, 1].label_outer()
axs[0, 1].set_title(r"numu")
axs[0, 1].text(1200, 0.15, "CC-COH", rotation=-90)
axs[0, 1].set_ylim(0, 0.5)
axs[0, 1].grid()
e_list, std_list = frac_e_vs_vtxX(numu_ccdis_ev)
axs[1, 1].plot(e_list, std_list)
axs[1, 1].label_outer()
axs[1, 1].text(1200, 0.15, "CC-DIS", rotation=-90)
axs[1, 1].set_ylim(0, 0.5)
axs[1, 1].grid()
e_list, std_list = frac_e_vs_vtxX(numu_ccqel_ev)
axs[2, 1].plot(e_list, std_list)
axs[2, 1].label_outer()
axs[2, 1].text(1200, 0.15, "CC-QEL", rotation=-90)
axs[2, 1].set_ylim(0, 0.5)
axs[2, 1].grid()
e_list, std_list = frac_e_vs_vtxX(numu_ccres_ev)
axs[3, 1].plot(e_list, std_list)
axs[3, 1].label_outer()
axs[3, 1].text(1200, 0.15, "CC-RES", rotation=-90)
axs[3, 1].set_ylim(0, 0.5)
axs[3, 1].grid()
e_list, std_list = frac_e_vs_vtxX(numu_ccmec_ev)
axs[4, 1].plot(e_list, std_list)
axs[4, 1].label_outer()
axs[4, 1].set(xlabel=r"Vertex x-position (cm)")
axs[4, 1].text(1200, 0.15, "CC-MEC", rotation=-90)
axs[4, 1].set_ylim(0, 0.5)
axs[4, 1].grid()
chipsnet.plotting.save(save_path + "vtx_vs_frac_energy")

In [ ]:
# Tidy up
del nuel_cccoh_ev
del nuel_ccdis_ev
del nuel_ccqel_ev
del nuel_ccres_ev
del nuel_ccmec_ev
del nuel_ccqelmec_ev
del numu_cccoh_ev
del numu_ccdis_ev
del numu_ccqel_ev
del numu_ccres_ev
del numu_ccmec_ev
del numu_ccqelmec_ev
del flux_ev

<a id='final'></a>

## Final performance and comparison with old reco/pid (final)
* 1
* 2
* 3

In [ ]:
def combined_process(config, data_name, cosmic_name, beam_name):
    # Get the dataframe from the dataset name
    events = chipsnet.utils.data_from_conf(config, data_name).testing_df(config.eval.examples)
    
    # Run the cosmic model inference
    events = chipsnet.utils.run_inference(
        events,
        chipsnet.utils.model_from_conf(config, cosmic_name),
        unstack=config.data.unstack,
        reco_pars=config.model.reco_pars
    )
    
    # Run the beam model inference
    events = chipsnet.utils.run_inference(
        events,
        chipsnet.utils.model_from_conf(config, beam_name),
        unstack=config.data.unstack,
        reco_pars=config.model.reco_pars
    )
    
    # Apply the event weights
    events = chipsnet.utils.apply_weights(
        events,
        total_num=config.eval.weights.total,
        nuel_frac=config.eval.weights.nuel,
        anuel_frac=config.eval.weights.anuel,
        numu_frac=config.eval.weights.numu,
        anumu_frac=config.eval.weights.anumu,
        cosmic_frac=config.eval.weights.cosmic,
        verbose=True,
    )

    # Apply the standard cuts
    events = chipsnet.utils.apply_standard_cuts(
        events,
        cosmic_cut=config.eval.cuts.cosmic,
        q_cut=config.eval.cuts.q,
        h_cut=config.eval.cuts.h,
        theta_cut=config.eval.cuts.theta,
        phi_cut=config.eval.cuts.phi,
        verbose=True,
    )
    
    # Combine beam categories into fully combined ones
    events = chipsnet.utils.full_comb_combine(events, "t_nu_nc_cat")
    
    # Run curve calculation
    curves_output = chipsnet.utils.calculate_curves(events, verbose=True)
    outputs = {
        "cuts": [curves_output["cuts"]],
        "sig_effs": [curves_output["sig_effs"]],
        "bkg_effs": [curves_output["bkg_effs"]],
        "purs": [curves_output["purs"]],
        "foms": [curves_output["foms"]],
        "fom_effs": [curves_output["fom_effs"]],
        "fom_purs": [curves_output["fom_purs"]],
        "sig_effs_auc": [curves_output["sig_effs_auc"]],
        "bkg_effs_auc": [curves_output["bkg_effs_auc"]],
        "pur_auc": [curves_output["pur_auc"]],
        "fom_auc": [curves_output["fom_auc"]],
        "roc_auc": [curves_output["roc_auc"]],
        "comb_matrices": [],
        "all_matrices": []
    }
    
    # TODO: THINK ABOUT HOW TO DO CLASSIFICATION CORRECTLY, not just >0.5
    # DO WE NEED TWO, ONE FOR COMB AND ONE MORE SPECIFIC
    class_prefix = "pred_t_comb_cat_"
    events["comb_cat_class"] = events.apply(
        chipsnet.utils.classify, axis=1, args=(3, class_prefix)
    )
    class_prefix = "pred_t_nu_nc_cat_"
    events["nu_nc_cat_class"] = events.apply(
        chipsnet.utils.classify, axis=1, args=(18, class_prefix),
    )
    
    # Calculate classification reports
    comb_report = classification_report(
        events["t_comb_cat"],
        events["comb_cat_class"],
        labels=[x for x in range(4)],
        target_names=chipsnet.data.MAP_FULL_COMB_CAT["labels"],
        sample_weight=events["w"],
        zero_division=0,
    )
    print(comb_report)
    cat_report = classification_report(
        events["t_nu_nc_cat"],
        events["nu_nc_cat_class"],
        labels=[x for x in range(19)],
        target_names=chipsnet.data.MAP_NU_NC_COMB_CAT["labels"],
        sample_weight=events["w"],
        zero_division=0,
    )
    print(cat_report)
    
    # Calculate confusion matrices
    matrix_comb = confusion_matrix(
        events["t_comb_cat"],
        events["comb_cat_class"],
        labels=[x for x in range(4)],
        sample_weight=events["w"],
        normalize="true",
    )
    matrix_comb = np.rot90(matrix_comb, 1)
    matrix_comb = pd.DataFrame(matrix_comb)
    outputs["comb_matrices"] = [matrix_comb]

    matrix_full = confusion_matrix(
        events["t_nu_nc_cat"],
        events["nu_nc_cat_class"],
        labels=[x for x in range(19)],
        sample_weight=events["w"],
        normalize="true",
    )
    matrix_full = np.rot90(matrix_full, 1)
    matrix_full = pd.DataFrame(matrix_full)
    outputs["all_matrices"] = [matrix_full]
    
    return events, outputs

In [ ]:
events, outputs = combined_process(config, "flux_cosmic", "cosmic_vtx", "beam_primaries")

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(
    outputs["comb_matrices"][0],
    annot=True, 
    vmin=0.0, 
    vmax=1.0,
    xticklabels=chipsnet.data.MAP_FULL_COMB_CAT["labels"],
    yticklabels=chipsnet.data.MAP_FULL_COMB_CAT["labels"][::-1],
    cbar=False,
    square=True,
    linewidth=1,
    linecolor="grey",
    cmap="Reds")
plt.tick_params(axis='both', which='both',
                bottom=False, top=False, left=False, right=False)
plt.xlabel('True category', fontsize = 30)
plt.ylabel('Predicted category', fontsize = 30)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
sns.heatmap(
    outputs["all_matrices"][0],
    annot=False, 
    vmin=0.0, 
    vmax=1.0,
    xticklabels=chipsnet.data.MAP_NU_NC_COMB_CAT["labels"],
    yticklabels=chipsnet.data.MAP_NU_NC_COMB_CAT["labels"][::-1],
    cbar=False,
    square=True,
    linewidth=1,
    linecolor="grey",
    cmap="Reds")
axs.tick_params(axis='both', which='both',
                bottom=False, top=False, left=False, right=False)
axs.set_xlabel('True category', fontsize = 30)
axs.set_ylabel('Predicted category', fontsize = 30)
axs.add_patch(patches.Rectangle((0, 13), 6, 6, linewidth=3, edgecolor='black', facecolor='none'))
axs.add_patch(patches.Rectangle((6, 7), 6, 6, linewidth=3, edgecolor='black', facecolor='none'))
axs.add_patch(patches.Rectangle((12, 1), 6, 6, linewidth=3, edgecolor='black', facecolor='none'))
axs.add_patch(patches.Rectangle((18, 0), 1, 1, linewidth=3, edgecolor='black', facecolor='none'))
plt.show()

In [ ]:
chipsnet.plotting.plot_curves([outputs], save_path+"final_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([outputs], events, save_path+"final_hists")

In [ ]:
chipsnet.plotting.plot_history_comparison(config, ["cosmic_vtx", "beam_primaries"], save_path+"final_history")

In [ ]:
# Tidy up
del final_ev, final_out

# Explainability and resilience

<a id='cosmic_calib'></a>

## Model calibration sensitivity: cosmic classification (cosmic_calib)
* 1
* 2
* 3

In [ ]:
ev_0, out_0 = chipsnet.utils.process_ds(config, "flux_cosmic", ["cosmic_vtx"])
ev_10, out_10 = chipsnet.utils.process_ds(config, "flux_cosmic_factor_10", ["cosmic_vtx"])
ev_20, out_20 = chipsnet.utils.process_ds(config, "flux_cosmic_factor_20", ["cosmic_vtx"])
ev_50, out_50 = chipsnet.utils.process_ds(config, "flux_cosmic_factor_50", ["cosmic_vtx"])

In [ ]:
# Tidy up
del ev_0, out_0
del ev_10, out_10
del ev_20, out_20
del ev_50, out_50

<a id='beam_calib'></a>

## Model calibration sensitivity: beam classification (beam_calib)
* 1
* 2
* 3

In [ ]:
ev_0, out_0 = chipsnet.utils.process_ds(config, "flux", ["beam_primaries"])
ev_10, out_10 = chipsnet.utils.process_ds(config, "flux_factor_10", ["beam_primaries"])
ev_20, out_20 = chipsnet.utils.process_ds(config, "flux_factor_20", ["beam_primaries"])
ev_50, out_50 = chipsnet.utils.process_ds(config, "flux_factor_50", ["beam_primaries"])

In [ ]:
chipsnet.plotting.plot_curves([out_0, out_10, out_20, out_50], save_path+"beam_calib_curves")

In [ ]:
chipsnet.plotting.plot_e_hists([out_0, out_10, out_20, out_50], ev_0, save_path+"beam_calib_hists")

In [ ]:
chipsnet.utils.print_output_comparison([out_0, out_10, out_20, out_50])

In [ ]:
# Tidy up
del ev_0, out_0
del ev_10, out_10
del ev_20, out_20
del ev_50, out_50

<a id='energy_calib'></a>

## Model calibration sensitivity: energy estimation (energy_calib)
* 1
* 2
* 3

In [ ]:
ev_0, out_0 = chipsnet.utils.predict_energies(config, "flux", ["nuel_cc_energy"])
ev_10, out_10 = chipsnet.utils.predict_energies(config, "flux_factor_10", ["nuel_cc_energy"])
ev_20, out_20 = chipsnet.utils.predict_energies(config, "flux_factor_20", ["nuel_cc_energy"])
ev_50, out_50 = chipsnet.utils.predict_energies(config, "flux_factor_50", ["nuel_cc_energy"])

In [ ]:
e_range = [[0, 8000], [0, 8000]]
e_bins = [40, 40]
fig, axs = plt.subplots(1, 4, figsize=(20, 4), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
axs[0].hist2d(v_cth_flux_ev[v_cth_flux_ev["t_comb_cat"] == 0]["t_nuEnergy"], v_cth_flux_ev[v_cth_flux_ev["t_comb_cat"] == 0]["nuel_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=v_cth_flux_ev[v_cth_flux_ev["t_comb_cat"] == 0]["w"], cmap="Reds")
axs[0].grid()
axs[0].label_outer()
axs[0].set(xlabel=r"True energy (MeV)", ylabel=r"Reco energy (MeV)")
axs[0].set_title(r"0")
axs[1].hist2d(v_cth_flux_10_ev[v_cth_flux_10_ev["t_comb_cat"] == 0]["t_nuEnergy"], v_cth_flux_10_ev[v_cth_flux_10_ev["t_comb_cat"] == 0]["nuel_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=v_cth_flux_10_ev[v_cth_flux_10_ev["t_comb_cat"] == 0]["w"], cmap="Reds")
axs[1].grid()
axs[1].label_outer()
axs[1].set(xlabel=r"True energy (MeV)")
axs[1].set_title(r"10")
axs[2].hist2d(v_cth_flux_20_ev[v_cth_flux_20_ev["t_comb_cat"] == 0]["t_nuEnergy"], v_cth_flux_20_ev[v_cth_flux_20_ev["t_comb_cat"] == 0]["nuel_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=v_cth_flux_20_ev[v_cth_flux_20_ev["t_comb_cat"] == 0]["w"], cmap="Reds")
axs[2].grid()
axs[2].label_outer()
axs[2].set(xlabel=r"True energy (MeV)")
axs[2].set_title(r"20")
axs[3].hist2d(v_cth_flux_50_ev[v_cth_flux_50_ev["t_comb_cat"] == 0]["t_nuEnergy"], v_cth_flux_50_ev[v_cth_flux_50_ev["t_comb_cat"] == 0]["nuel_cc_energy_pred_t_nuEnergy"],
              range=e_range, bins=e_bins, weights=v_cth_flux_50_ev[v_cth_flux_50_ev["t_comb_cat"] == 0]["w"], cmap="Reds")
axs[3].grid()
axs[3].label_outer()
axs[3].set(xlabel=r"True energy (MeV)")
axs[3].set_title(r"50")
chipsnet.plotting.save(save_path + "true_vs_reco_energy_comb")

In [ ]:
v_cth_flux_ev['true_reco_frac_nuEnergy'] = (v_cth_flux_ev["t_nuEnergy"] - v_cth_flux_ev["nuel_cc_energy_pred_t_nuEnergy"]) / v_cth_flux_ev["t_nuEnergy"]
v_cth_flux_10_ev['true_reco_frac_nuEnergy'] = (v_cth_flux_10_ev["t_nuEnergy"] - v_cth_flux_10_ev["nuel_cc_energy_pred_t_nuEnergy"]) / v_cth_flux_10_ev["t_nuEnergy"]
v_cth_flux_20_ev['true_reco_frac_nuEnergy'] = (v_cth_flux_20_ev["t_nuEnergy"] - v_cth_flux_20_ev["nuel_cc_energy_pred_t_nuEnergy"]) / v_cth_flux_20_ev["t_nuEnergy"]
v_cth_flux_50_ev['true_reco_frac_nuEnergy'] = (v_cth_flux_50_ev["t_nuEnergy"] - v_cth_flux_50_ev["nuel_cc_energy_pred_t_nuEnergy"]) / v_cth_flux_50_ev["t_nuEnergy"]

In [ ]:
e_range = [[0, 8000], [0, 8000]]
e_bins = [40, 40]
fig, axs = plt.subplots(1, 4, figsize=(20, 4), gridspec_kw={"hspace": 0.1, "wspace": 0.1})
e_list, std_list = frac_e_vs_e(v_cth_flux_ev)
axs[0].plot(e_list, std_list)
axs[0].grid()
axs[0].label_outer()
axs[0].set(xlabel=r"True energy (MeV)", ylabel=r"Reco energy (MeV)")
axs[0].set_title(r"0")
axs[0].set_ylim(0, 0.5)
e_list, std_list = frac_e_vs_e(v_cth_flux_10_ev)
axs[1].plot(e_list, std_list)
axs[1].grid()
axs[1].label_outer()
axs[1].set(xlabel=r"True energy (MeV)")
axs[1].set_title(r"10")
axs[1].set_ylim(0, 0.5)
e_list, std_list = frac_e_vs_e(v_cth_flux_20_ev)
axs[2].plot(e_list, std_list)
axs[2].grid()
axs[2].label_outer()
axs[2].set(xlabel=r"True energy (MeV)")
axs[2].set_title(r"20")
axs[2].set_ylim(0, 0.5)
e_list, std_list = frac_e_vs_e(v_cth_flux_50_ev)
axs[3].plot(e_list, std_list)
axs[3].grid()
axs[3].label_outer()
axs[3].set(xlabel=r"True energy (MeV)")
axs[3].set_title(r"50")
axs[3].set_ylim(0, 0.5)
chipsnet.plotting.save(save_path + "true_vs_reco_energy_comb")

In [ ]:
# Tidy up
del ev_0, out_0
del ev_10, out_10
del ev_20, out_20
del ev_50, out_50

<a id='cosmic_cluster'></a>

## Clustering: cosmic classification
* 1
* 2
* 3

<a id='beam_cluster'></a>

## Clustering: beam classification
* 1
* 2
* 3

In [ ]:
u_ev, u_out = chipsnet.utils.process_ds(config, "uniform", ["beam_primaries"])

In [ ]:
pca_model = chipsnet.utils.model_from_conf(config, "beam_primaries")
u_ev = chipsnet.utils.run_pca(u_ev, pca_model, verbose=True, components=2, standardise=True)
cat_0_ev = u_ev[u_ev.t_comb_cat == 0]
cat_1_ev = u_ev[u_ev.t_comb_cat == 1]
cat_2_ev = u_ev[u_ev.t_comb_cat == 2]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 8))
axs.margins(0.05) # Optional, just adds 5% padding to the autoscaling
axs.plot(cat_0_ev.pca_0, cat_0_ev.pca_1, marker='o', linestyle='', ms=12, label="nuel-cc")
axs.plot(cat_1_ev.pca_0, cat_1_ev.pca_1, marker='o', linestyle='', ms=12, label="numu-cc")
axs.plot(cat_2_ev.pca_0, cat_2_ev.pca_1, marker='o', linestyle='', ms=12, label="nc")
axs.legend()
axs.set_xlabel('pca-0', fontsize=17)
axs.set_ylabel('pca-1', fontsize=17)
chipsnet.plotting.save(save_path + "pca")

In [ ]:
tsne_model = chipsnet.utils.model_from_conf(config, "beam_primaries")
u_ev = chipsnet.utils.run_tsne(u_ev, tsne_model, components=2, standardise=True, max_events=10000)
cat_0_ev = u_ev[u_ev.t_comb_cat == 0]
cat_1_ev = u_ev[u_ev.t_comb_cat == 1]
cat_2_ev = u_ev[u_ev.t_comb_cat == 2]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 8))
axs.margins(0.05) # Optional, just adds 5% padding to the autoscaling
axs.plot(cat_0_ev.tsne_0, cat_0_ev.tsne_1, marker='o', linestyle='', ms=12, label="nuel-cc")
axs.plot(cat_1_ev.tsne_0, cat_1_ev.tsne_1, marker='o', linestyle='', ms=12, label="numu-cc")
axs.plot(cat_2_ev.tsne_0, cat_2_ev.tsne_1, marker='o', linestyle='', ms=12, label="nc")
axs.legend()
axs.set_xlabel('tsne-0', fontsize=17)
axs.set_ylabel('tsne-1', fontsize=17)
chipsnet.plotting.save(save_path + "tsne")

In [ ]:
'''
colors = ["#004d00", "#009900", "#00e600", "#33ff33", 
          "#00004d", "#0000b3", "#1a1aff", "#8080ff", 
          "#660000", "#cc0000", "#ff3333", "#ff9999",
          "#660000", "#cc0000", "#ff3333", "#ff9999"]
plt.figure(figsize=(16,8))
sns.scatterplot(x="beam-tsne-0", y="beam-tsne-1", hue=u_eval.b_model.cat, palette=sns.color_palette(colors), data=beam_data, legend="full", alpha=0.3)
plt.show()
'''

In [ ]:
# Tidy up
del u_ev, u_out

<a id='cosmic_explain'></a>

## Explanation: cosmic classification
* 1
* 2
* 3

<a id='beam_explain'></a>

## Explanation: beam classification
* 1
* 2
* 3

In [ ]:
explain_ev, explain_out = chipsnet.utils.process_ds(config, "stacked_noreco")

In [ ]:
grad_cam_model = chipsnet.utils.model_from_conf(config, "explain")
grad_cams_1 = chipsnet.utils.explain_gradcam(explain_ev, grad_cam_model, 10, output="t_nu_nc_cat", layer_name="block0_path0_conv1")
grad_cams_2 = chipsnet.utils.explain_gradcam(explain_ev, grad_cam_model, 10, output="t_nu_nc_cat", layer_name="block1_path0_conv1")
grad_cams_3 = chipsnet.utils.explain_gradcam(explain_ev, grad_cam_model, 10, output="t_nu_nc_cat", layer_name="block2_conv1")

In [ ]:
event = 2
fig, axs = plt.subplots(1, 3, figsize=(15, 12), gridspec_kw={'hspace': 0.3})
plt.setp(axs, xticks=[0, 16, 32, 48, 64], yticks=[0, 16, 32, 48, 64])
axs[0].imshow(grad_cams_1[event], cmap='Reds', origin='lower')
axs[0].set(xlabel='$\phi$ bins', ylabel='$\\theta$ bins', title='GradCam Block 1')
axs[1].imshow(grad_cams_2[event], cmap='Reds', origin='lower')
axs[1].set(xlabel='$\phi$ bins', ylabel='$\\theta$ bins', title='GradCam Block 2')
axs[2].imshow(grad_cams_3[event], cmap='Reds', origin='lower')
axs[2].set(xlabel='$\phi$ bins', ylabel='$\\theta$ bins', title='GradCam Block 3')
chipsnet.plotting.save(save_path + "grad_cam")

In [ ]:
activation_model = chipsnet.utils.model_from_conf(config, "explain")
activations_1 = chipsnet.utils.explain_activation(explain_ev, activation_model, 10, output="t_nu_nc_cat", layer_name="block0_path0_conv1")
activations_2 = chipsnet.utils.explain_activation(explain_ev, activation_model, 10, output="t_nu_nc_cat", layer_name="block1_path0_conv1")
activations_3 = chipsnet.utils.explain_activation(explain_ev, activation_model, 10, output="t_nu_nc_cat", layer_name="block2_conv1")

In [ ]:
event = 1
fig, axs = plt.subplots(1, 3, figsize=(20, 8))
plt.rc('grid', linestyle="-", color='black')

axs[0].set_xticks([0, 32, 64, 96, 128, 160, 192, 224, 256])
axs[0].set_yticks([0, 32, 64, 96, 128, 160, 192, 224, 256])
axs[0].set_xticklabels('')
axs[0].set_yticklabels('')
extent = (0, 256, 256, 0)
axs[0].imshow(activations_1[event], cmap='Reds', origin='lower', extent=extent)
axs[0].set_title('Block 1 Activations')
axs[0].grid()

extent = (0, 192, 192, 0)
axs[1].set_xticks([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192])
axs[1].set_yticks([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192])
axs[1].set_xticklabels('')
axs[1].set_yticklabels('')
axs[1].imshow(activations_2[event], cmap='Reds', origin='lower', extent=extent)
axs[1].set_title('Block 2 Activations')
axs[1].grid()

extent = (0, 128, 128, 0)
axs[2].set_xticks([0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128])
axs[2].set_yticks([0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128])
axs[2].set_xticklabels('')
axs[2].set_yticklabels('')
axs[2].imshow(activations_3[event], cmap='Reds', origin='lower', extent=extent)
axs[2].set_title('Block 3 Activations')
axs[2].grid()

chipsnet.plotting.save(save_path + "activations")

In [ ]:
# Tidy up
del explain_ev, explain_out

<a id='energy_explain'></a>

## Explanation: energy estimation
* 1
* 2
* 3